## **Installation of libraries**

First thing to do is install the two libraries for reading and writing data to and from ADX.

In [ ]:
!pip install azure-kusto-data

In [ ]:
!pip install azure-kusto-ingest

## **Documentation**

https://docs.microsoft.com/en-us/azure/kusto/api/python/kusto-python-client-library
https://github.com/Azure/azure-kusto-python

https://github.com/Azure/azure-kusto-python

## **Import Libraries**

In [ ]:
from datetime import timedelta

from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.kusto.data.request import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties

## **Connect to the ADX cluster**

In [ ]:
cluster = "https://mdrnext.westeurope.kusto.windows.net"

In [ ]:
kcsb = KustoConnectionStringBuilder.with_aad_device_authentication(cluster)

In [ ]:
client = KustoClient(kcsb)

## **Define the query**

In [ ]:
db = "mdr-signals"
query = "signals | take 10"

## **Execute the query**

In [ ]:
response = client.execute(db, query)

## **Handover to Pandas Dataframe**

In [ ]:
df = dataframe_from_result_table(response.primary_results[0])

In [ ]:
display(df)

##  **Data Ingestion to ADX**

In [ ]:
from azure.kusto.ingest import (
    KustoIngestClient,
    IngestionProperties,
    FileDescriptor,
    BlobDescriptor,
    StreamDescriptor,
    DataFormat,
    ReportLevel,
    IngestionMappingType,
    KustoStreamingIngestClient,
)

## **Setting Properties for Ingest**

In [ ]:
ingestion_props = IngestionProperties(
    database="Playground_WS",
    table="signals_cr",
    dataFormat=DataFormat.CSV,
    # in case status update for success are also required
    reportLevel=ReportLevel.FailuresAndSuccesses
    # in case a mapping is required
    # ingestionMappingReference="{json_mapping_that_already_exists_on_table}"
    # ingestionMappingType=IngestionMappingType.Json
)

## **Choosing cluster and connecting**

In [ ]:
cluster_in = "https://ingest-mdrnext.westeurope.kusto.windows.net"

In [ ]:
kcsb_in = KustoConnectionStringBuilder.with_aad_device_authentication(cluster_in)

In [ ]:
client_in = KustoIngestClient(kcsb_in)

## **Ingesting data into previously created table**

In [ ]:
client_in.ingest_from_dataframe(df, ingestion_properties=ingestion_props)